In [1]:
import os

In [2]:
%pwd

'd:\\Nikku\\End to End Project\\Healthy.ai\\research\\Food image Classification'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Nikku\\End to End Project\\Healthy.ai'

## Update the entity

In [6]:
from pathlib import  Path
from dataclasses import dataclass


@dataclass(frozen=True)
class FoodTrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

## Update the Configuration Manager

In [7]:
from Healthyai.constants import *
from Healthyai.utils.common import read_yaml, create_directories
import tensorflow as tf

[2024-02-22 15:45:08,841: WARNING: module_wrapper: From c:\Users\akash\anaconda3\envs\healthyai\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config =  read_yaml(config_filepath)
        self.params =  read_yaml(params_filepath)
        
        
        create_directories([self.config.artifacts])
        
    def get_food_model_training_config(self) -> FoodTrainingConfig:
        training = self.config.food_image_model_training
        prepare_base_model = self.config.food_image_prepare_base_model
        params = self.params.VGG16
        training_data = os.path.join(self.config.food_image_data_ingestion.unzip_dir, "food classification")
        create_directories([Path(training.root_dir)])
        
        
        training_config = FoodTrainingConfig(
            root_dir= Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config

# Update the Components

In [9]:
import os
import urllib.request as request
from zipfile import  ZipFile
import tensorflow as tf
import time

In [10]:
class FoodTraining:
    def __init__(self, config: FoodTrainingConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        valid_datagenerator  = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        ) 
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
            
        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        
        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
        )
        
        self.save_model(
            path=self.config.trained_model_path,
            model = self.model
        )
    

In [11]:
# Pipeline
try:
    config = ConfigurationManager()
    traing_config = config.get_food_model_training_config()
    training = FoodTraining(config=traing_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2024-02-22 15:45:09,766: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-22 15:45:09,768: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-22 15:45:09,768: INFO: common: Created directory at : artifacts]
[2024-02-22 15:45:09,769: INFO: common: Created directory at : artifacts\food_image_classification\training]
[2024-02-22 15:45:09,894: WARNING: module_wrapper: From c:\Users\akash\anaconda3\envs\healthyai\lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
[2024-02-22 15:45:09,929: WARNING: module_wrapper: From c:\Users\akash\anaconda3\envs\healthyai\lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.
]
Found 1980 images belonging to 11 classes.
Found 7920 images belonging to 11 classes.
Epoch 1/10
[2024-02-22 15:45:11,062: WARNI

c:\Users\akash\anaconda3\envs\healthyai\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
